# Análise dos dados do Spotify da conta pessoal

Esse notebook tem o intuito de ler, tratar os dados e gerar a base de dados que será lida dentro do PowerBI

## Configurações iniciais

### Instala bibliotecas não instaladas por padrão

In [0]:
%pip install openpyxl

### Importe das bibliotecas

In [0]:
import pandas as pd
from pyspark.sql.functions import col
import json

### Cria o banco de dados

In [0]:
%sql

CREATE TABLE IF NOT EXISTS sandbox_prd.analytics.tb_user_spotify (
  
)

## Carregamento das fontes de dados do volumes

#### Histórico extendido

Descrição dos arquivos: </br>
Os arquivos de histórico extendido do Spotify contêm registros detalhados das atividades de streaming do usuário, incluindo músicas e vídeos reproduzidos ao longo dos anos. Cada arquivo representa um período específico e inclui informações como data e hora da reprodução (`ts`), nome da faixa, artista, plataforma utilizada, tipo de conteúdo (música, podcast, vídeo), e outros metadados relevantes. Esses dados permitem analisar padrões de consumo, preferências musicais e comportamento do usuário na plataforma Spotify.

In [0]:
df_2021_2023_audio = pd.read_json("/Volumes/sandbox_prd/analytics/files/spotify/me/extended/Streaming_History_Audio_2021-2023_0.json")

df_2023_2024_audio = pd.read_json("/Volumes/sandbox_prd/analytics/files/spotify/me/extended/Streaming_History_Audio_2023-2024_1.json")

df_2024_2025_audio = pd.read_json("/Volumes/sandbox_prd/analytics/files/spotify/me/extended/Streaming_History_Audio_2024-2025_2.json")

df_2022_2025_video = pd.read_json("/Volumes/sandbox_prd/analytics/files/spotify/me/extended/Streaming_History_Video_2022-2025.json")

#### Arquivos da conta

Descrição: </br> 
Os arquivos da conta do Spotify contêm o histórico de reprodução do usuário, separados por tipo de conteúdo, como músicas e podcasts. Cada arquivo registra informações detalhadas sobre as faixas ou episódios reproduzidos, incluindo data e hora da reprodução, nome da faixa ou episódio, artista ou criador, duração, e outros metadados relevantes. Esses dados permitem analisar o comportamento de consumo, preferências e padrões de uso do usuário na plataforma Spotify.

In [0]:
df_stremingHistoricalMusic = pd.read_json("/Volumes/sandbox_prd/analytics/files/spotify/me/account/StreamingHistory_music_0.json")
df_stremingHistoricalPodcast = pd.read_json("/Volumes/sandbox_prd/analytics/files/spotify/me/account/StreamingHistory_podcast_0.json")

## Tratamento dos dados

### Empilha os arquivos de histórico extendido

In [0]:
df_audio = pd.concat([df_2021_2023_audio, df_2023_2024_audio, df_2024_2025_audio])

### Remove colunas desnecessárias

In [0]:
columns_to_drop = [
  "conn_country",
  "ip_addr",
  "spotify_episode_uri",
  "audiobook_title",
  "audiobook_uri",
  "audiobook_chapter_uri",
  "audiobook_chapter_title"
  ]

df_data_audio_clean = df_audio.drop(columns=columns_to_drop)